In [16]:
import os

# Create main folder
project_folder = "wallet-risk-scoring"
os.makedirs(project_folder, exist_ok=True)

# Create required files/subfolders
open(os.path.join(project_folder, "main.py"), 'a').close()
open(os.path.join(project_folder, "requirements.txt"), 'a').close()
open(os.path.join(project_folder, "wallets.csv"), 'a').close()
open(os.path.join(project_folder, "features.csv"), 'a').close()
open(os.path.join(project_folder, "explanation.md"), 'a').close()

print(" Folder structure created in your Jupyter Notebook directory.")


 Folder structure created in your Jupyter Notebook directory.


In [14]:


file_path = "wallet-risk-scoring/features.csv"
print("File exists:", os.path.exists(file_path))
print("File size in bytes:", os.path.getsize(file_path))


File exists: True
File size in bytes: 0


In [12]:
import pandas as pd

wallets = pd.read_csv("wallets.csv")
print(wallets.columns.tolist())


['wallet_id']


In [12]:

# Load wallet list
wallets = pd.read_csv("wallets.csv")

# Clean column names
wallets.columns = wallets.columns.str.strip().str.lower().str.replace(" ", "_")

# Check available columns
print("Available columns:", wallets.columns.tolist())

# Use the actual column name
wallet_list = wallets["wallet_id"].tolist()

# Create output directory
os.makedirs("wallet-risk-scoring", exist_ok=True)


Available columns: ['wallet_id']


In [18]:
import requests
import json
import time

def get_mock_compound_data(wallet_address):
    # Simulating API response
    return {
        "wallet": wallet_address,
        "supply_count": 5,
        "borrow_count": 2,
        "liquidation_count": 1,
        "total_supply_usd": 1050,
        "total_borrow_usd": 800
    }




In [20]:
# Loop through wallets and fetch + save data
for i, wallet in enumerate(wallet_list):
    print(f"Fetching data for wallet {i+1}/{len(wallet_list)}: {wallet}")

    # Fetch data (replace this with real API call later)
    data = get_mock_compound_data(wallet)

    # Save to JSON file
    with open(f"wallet-risk-scoring/{wallet}.json", "w") as f:
        json.dump(data, f)

    time.sleep(0.5)  # Rate limit friendly


Fetching data for wallet 1/103: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Fetching data for wallet 2/103: 0x06b51c6882b27cb05e712185531c1f74996dd988
Fetching data for wallet 3/103: 0x0795732aacc448030ef374374eaae57d2965c16c
Fetching data for wallet 4/103: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Fetching data for wallet 5/103: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Fetching data for wallet 6/103: 0x104ae61d8d487ad689969a17807ddc338b445416
Fetching data for wallet 7/103: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Fetching data for wallet 8/103: 0x124853fecb522c57d9bd5c21231058696ca6d596
Fetching data for wallet 9/103: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Fetching data for wallet 10/103: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Fetching data for wallet 11/103: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
Fetching data for wallet 12/103: 0x19df3e87f73c4aaf4809295561465b993e102668
Fetching data for wallet 13/103: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
Fetching data for wal

In [38]:


def calculate_risk_score(wallet_data):
    supply_count = wallet_data.get("supply_count", 0)
    borrow_count = wallet_data.get("borrow_count", 0)
    liquidation_count = wallet_data.get("liquidation_count", 0)
    total_supply = wallet_data.get("total_supply_usd", 0)
    total_borrow = wallet_data.get("total_borrow_usd", 0)

    # Derived feature
    borrow_ratio = total_borrow / (total_supply + 1)

# Risk scoring formula
    risk_score = (
        borrow_ratio * 400 +
        liquidation_count * 200 +
        borrow_count * 100 -
        supply_count * 50
    )

# Clip score between 0 and 1000
    risk_score = max(0, min(1000, risk_score))

    return {
        "supply_count": supply_count,
        "borrow_count": borrow_count,
        "liquidation_count": liquidation_count,
        "total_supply_usd": total_supply,
        "total_borrow_usd": total_borrow,
        "borrow_ratio": borrow_ratio,
        "risk_score": round(risk_score, 2)
    }

In [42]:
# Directory containing JSON files
json_dir = "wallet-risk-scoring"

results = []

# Loop through JSON files
for filename in os.listdir(json_dir):
    if filename.endswith(".json"):
        wallet_address = filename.replace(".json", "")
        with open(os.path.join(json_dir, filename), "r") as f:
            wallet_data = json.load(f)
            features = calculate_risk_score(wallet_data)
            features["wallet_address"] = wallet_address
            results.append(features)

# Save final DataFrame
df = pd.DataFrame(results)
df.to_csv("wallet_risk_scores.csv", index=False)

print(" Risk scores saved to wallet_risk_scores.csv")

 Risk scores saved to wallet_risk_scores.csv


In [46]:


# Load the full scores file
df = pd.read_csv("wallet_risk_scores.csv")

# Rename columns for submission format
df_submission = df[["wallet_address", "risk_score"]].rename(columns={
    "wallet_address": "wallet_id",
    "risk_score": "score"
})

# Save submission CSV
df_submission.to_csv("submission_wallet_scores.csv", index=False)

print("Final submission CSV saved as submission_wallet_scores.csv")


Final submission CSV saved as submission_wallet_scores.csv


In [10]:
# Save submission CSV
df_submission.to_csv("submission_wallet_scores.csv", index=False)

print(" Final submission CSV saved as submission_wallet_scores.csv")

 Final submission CSV saved as submission_wallet_scores.csv
